In [98]:
import pandas as pd
import numpy as np

import json
from rdkit import Chem
import pandas as pd
import itertools
import sys
from tqdm import tqdm 
from itertools import chain, combinations
import sys
import multiprocessing as mp
from functools import partial
from collections import Counter
tqdm.pandas()
sys.path.append('../')
from common import *

In [99]:
num = 50
cut_off = 0.3
cut_off_retrosynthesis_path = f'../../../Data_retrosynthesis/not_lipid_filter20percent/top{num}_{cut_off}_add_no_ec_re/'
cut_off_path = f'../../../Results/not_lipid_filter20percent/top{num}_{cut_off}_add_no_ec_re/'
ymdb_output_final = '../../../Data/ymdb/ymdb_final_result.xlsx'
mnxmeta_smile_inchikey_dict_path = '../../../Data/database/mnxmeta_smile_inchikey_dict.json'

YMDB_success_met_smile_uptake_file = cut_off_path + f'YMDB_success_met_smile_top{num}_{cut_off}.pickle'
not_lipid_YMDB_fail_met_smile_uptake_file = cut_off_path + f'YMDB_fail_met_smile_top{num}_{cut_off}.pickle'

not_lipid_rxndb_met_max_score_file = cut_off_path + f'rxndb_met_max_score_pd_top{num}_{cut_off}.csv'
uptake_met_path = '../../../Data/ymdb/ymdb_uptake.csv'
not_lipid_rxndb_inteme_only_one_new_target_path = cut_off_path + 'RXNdb_inteme_only_one_new_target_top50.csv'
# total_met_inchikey0_file = '../../../Results/analysis/total_met_inchikey0.pickle'
total_met_inchikey0_file = '../../../Results/analysis/total_met_inchikey0_filter.pickle'
yeast_model_path = '../../../Data/model/yeast-GEM-final.csv'
sink0_path = cut_off_path + 'sink0_new_met_only_new_one_top50.pickle'
not_lipid_YMDB_fail_met_smile_final_file = cut_off_path + f'YMDB_fail_met_smile_final_top{num}_{cut_off}.pickle'
not_lipid_fail_smiles2metnetx_path = cut_off_path + f'fail_smiles2metnetx_top{num}_{cut_off}.pickle'
not_lipid_YMDB_fail_met_smile_final_expand_file = cut_off_path + f'YMDB_fail_met_smile_final_expand_top{num}_{cut_off}.pickle'
not_lipid_YMDB_fail_met_smile_final_expand_txt_file = cut_off_path + f'YMDB_fail_met_smile_final_expand_top{num}_{cut_off}.txt'
not_lipid_YMDB_fail_met_smile_final_expand_txt_file_ = cut_off_path + f'YMDB_fail_met_smile_final_expand_top{num}_{cut_off}_.txt'
sink_inchi_path = cut_off_path + 'sink_inchi.csv'
sink_smiles_path = cut_off_path + 'sink_smiles.csv'

In [100]:
# with open(met_path, 'r') as f:
#     met_inchikey = json.load(f)
# print('met_inchikey', len(met_inchikey))

In [101]:
# sink0 = load_pickle(sink0_path)
# len(sink0)

In [102]:
def myfunc_new(index_row,sink0,total_inchikey0):
    row = index_row[1]
    reactant_smile = row['reactant_inchikey0']
    productsmile = row['product_inchikey0'].split('.')
    reactant_smile = ast.literal_eval(reactant_smile)
    productsmile_in_total = []
    add_inchikey0 = []
    for i in productsmile:
        try:  
            if i in sink0:
                productsmile_in_total.append(i)
        except:
            continue
    if len(productsmile_in_total) == len(productsmile):
        for i in reactant_smile:
            if i in total_inchikey0:
                add_inchikey0.append(i)
    return add_inchikey0
def myfunc_reverse_new(index_row,sink0,total_inchikey0):
    row = index_row[1]
    reactant_smile = row['reactant_inchikey0']
    productsmile = row['product_inchikey0'].split('.')
    reactant_smile = ast.literal_eval(reactant_smile)
    reactant_smile_in_total = []
    add_inchikey0 = []
    for i in reactant_smile:
        try:   #    
            if i not in total_inchikey0:
                reactant_smile_in_total.append(i)
            else:
                if i in sink0:
                    reactant_smile_in_total.append(i)
        except:
            continue
    if len(reactant_smile_in_total) == len(reactant_smile):
        for i in productsmile:
            add_inchikey0.append(i)
    return add_inchikey0

In [103]:
yeast8_total_smiles = get_all_smiles_in_model(yeast_model_path,ymdb_output_final)

success_smiles = load_pickle(YMDB_success_met_smile_uptake_file)
uptake_met = pd.read_csv(uptake_met_path)   
uptake_smiles = uptake_met['SMILES'].to_list()
sink_smiles = yeast8_total_smiles + success_smiles + uptake_smiles
sink0 = [smiles2inchikey0(i) for i in sink_smiles]
not_lipid_rxndb_met_max_score = pd.read_csv(not_lipid_rxndb_met_max_score_file)
not_lipid_rxndb_met_max_score = not_lipid_rxndb_met_max_score[not_lipid_rxndb_met_max_score['score']==1]
not_lipid_rxndb_met_max_smiles = not_lipid_rxndb_met_max_score['smile'].tolist()
not_lipid_rxndb_met_max_inchikey0 = [smiles2inchikey0(i) for i in not_lipid_rxndb_met_max_smiles]
sink0 = sink0 + not_lipid_rxndb_met_max_inchikey0
sink0 = list(set(sink0))
print(len(sink0))

1494


In [104]:
not_lipid_rxndb_inteme_only_one_new = pd.read_csv(not_lipid_rxndb_inteme_only_one_new_target_path)
print(not_lipid_rxndb_inteme_only_one_new.shape)
# sink0 = load_pickle(sink0_path)

(492, 13)


In [105]:
not_lipid_rxndb_inteme_only_one_new.head(3)

,Unnamed: 0,EC number,rule,templateID,templateSubstrate,rxn_smiles_basic,rxn_smiles_final,reactant_smile,productsmile,similarity,reactant_inchikey0,product_inchikey0,new_num
0,18,3.1.3.25,[C:1]-[OH;D1;+0:2].[O;-;D1;H0:4]-[P;H0;D4;+0:3...,MNXR151315_reverse,['O[C@H]1[C@H](O)[C@@H](O)[C@H](O)[C@@H](O)[C@...,O.O=P([O-])([O-])OC1C(O)C(O)C(OP(=O)([O-])[O-]...,O.O=P([O-])([O-])OC1C(O)C(O)C(OP(=O)([O-])[O-]...,O.O=P([O-])([O-])OC1C(O)C(O)C(OP(=O)([O-])[O-]...,O=P([O-])([O-])OC1C(O)C(O)C(O)C(O)C1O.O=P([O-]...,"[0.6296296296, 1]","['PELZSPZCXGTUMR', 'XLYOFNOQVPJJNP']",INAPMGSXUVUWAF.NBIIXXVUZAFLBC,1
1,19,3.1.3.25,[C:1]-[OH;D1;+0:2].[O;-;D1;H0:4]-[P;H0;D4;+0:3...,MNXR151315_reverse,['O[C@H]1[C@H](O)[C@@H](O)[C@H](O)[C@@H](O)[C@...,O.O=P([O-])([O-])OC1C(O)C(O)C(O)C(O)C1OP(=O)([...,O.O=P([O-])([O-])OC1C(O)C(O)C(O)C(O)C1OP(=O)([...,O.O=P([O-])([O-])OC1C(O)C(O)C(O)C(O)C1OP(=O)([...,O=P([O-])([O-])OC1C(O)C(O)C(O)C(O)C1O.O=P([O-]...,"[0.6296296296, 1]","['MCKAJXMRULSUKI', 'XLYOFNOQVPJJNP']",INAPMGSXUVUWAF.NBIIXXVUZAFLBC,1
2,28,3.1.3.25,[C:1]-[OH;D1;+0:2].[O;-;D1;H0:4]-[P;H0;D4;+0:3...,MNXR151315_reverse,['O[C@H]1[C@H](O)[C@@H](O)[C@H](O)[C@@H](O)[C@...,O.O=P([O-])([O-])OC1C(O)C(OP(=O)([O-])[O-])C(O...,O.O=P([O-])([O-])OC1C(O)C(OP(=O)([O-])[O-])C(O...,O.O=P([O-])([O-])OC1C(O)C(OP(=O)([O-])[O-])C(O...,O=P([O-])([O-])OC1C(O)C(O)C(OP(=O)([O-])[O-])C...,"[0.6071428571, 1]","['XLYOFNOQVPJJNP', 'ZAWIXNGTTZTBKV']",MMWCIQZXVOZEGG.NBIIXXVUZAFLBC,1


In [106]:
not_lipid_target_smiles_filter20percent_else_path = '../../../Results/not_lipid_filter20percent/target_smiles_not_lipid_filter20percent_else.pickle'
not_lipid_target_smiles_filter20percent_else = load_pickle(not_lipid_target_smiles_filter20percent_else_path)
filter_inchi = [smiles2inchikey0(x) for x in not_lipid_target_smiles_filter20percent_else]

In [121]:
print(len(sink0))
sink0 = [x for x in sink0 if x not in filter_inchi]
print(len(sink0))

1494
1277


In [122]:
# not_lipid_rxndb_inteme_only_one_new['reactant_inchikey0'] = not_lipid_rxndb_inteme_only_one_new['reactant_inchikey0'].apply(lambda x:literal_eval(x))

success_predict_met_num = 0
filter_inchi = [smiles2inchikey0(x) for x in not_lipid_target_smiles_filter20percent_else]
filter_inchi

for x in not_lipid_rxndb_inteme_only_one_new['reactant_inchikey0']:
    if all(inchi not in filter_inchi for inchi in x):
        print(1)

In [108]:
# def pool_func(df,sink0,total_inchikey0):
#     add_inchikey0 = list(map(partial(myfunc_new,sink0=sink0,total_inchikey0 = total_inchikey0),df.iterrows()))
#     add_inchikey0_ = list(map(partial(myfunc_reverse_new,sink0=sink0,total_inchikey0 = total_inchikey0),df.iterrows()))
#     #flatten
#     add_inchikey0 = [j for i in add_inchikey0 for j in i]
#     add_inchikey0_ = [j for i in add_inchikey0_ for j in i]
#     return (add_inchikey0,add_inchikey0_)

In [123]:
total_inchikey0 = load_pickle(total_met_inchikey0_file)

In [110]:
# chunks = np.array_split(not_lipid_rxndb_inteme_only_one_new, 100)
# with mp.Pool(40) as pool:
#     results = list(tqdm(pool.imap(partial(pool_func,sink0 = sink0,total_inchikey0 = total_inchikey0), chunks), total=len(chunks)))
# for i in results:
#         if i[0]:
#             sink0 += i[0]
#         if i[1]:
#             sink0 += i[1]
# sink0 = list(set(sink0))
# print(len(sink0))

In [111]:
# dump_file(sink0, sink0_path)

In [112]:
# def calculate_success_fail_target_smiles(target_smiles_file,sink0,uptake_met_path):
#     target_smiles = load_pickle(target_smiles_file)
#     uptake_smiles = pd.read_csv(uptake_met_path)['SMILES'].tolist()
#     uptake_inchikey0 = [smiles2inchikey0(i) for i in uptake_smiles]
#     target_smiles = [i for i in target_smiles if smiles2inchikey0(i) not in uptake_inchikey0]
#     # smiles_success = smiles_success + uptake_smiles
#     success_target_smiles = []
#     for i in tqdm(target_smiles,total=len(target_smiles)):
#         if smiles2inchikey0(i) in sink0:
#             success_target_smiles.append(i)
            
#     print('success number:',len(success_target_smiles))
#     fail_target_smiles = [i for i in target_smiles if i not in success_target_smiles]
#     print('fail number:',len(fail_target_smiles))
#     return success_target_smiles,fail_target_smiles

In [113]:
# success_smiles,fail_smiles = calculate_success_fail_target_smiles(not_lipid_YMDB_fail_met_smile_uptake_file,sink0,uptake_met_path)

## 遍历每一行 如果只有一个inchi不在sink0里面  就是成功筛选的反应，其中不在sink0里面的inchi就是新代谢物

In [126]:
print(not_lipid_rxndb_inteme_only_one_new.shape)
# not_lipid_rxndb_inteme_only_one_new['reactant_inchikey0'] = not_lipid_rxndb_inteme_only_one_new['reactant_inchikey0'].apply(lambda x:literal_eval(x))
new_reaction_num = 0
new_met = []
for index,row in tqdm(not_lipid_rxndb_inteme_only_one_new.iterrows()):
    inchikey_list = []
    inchikey_list +=row['reactant_inchikey0']
    inchikey_list +=row['product_inchikey0'].split('.')
    # print(len([x for x in inchikey_list if x not in sink0]))
    if len([x for x in inchikey_list if x not in sink0])==1:
        new_reaction_num +=1
        new_met += [x for x in inchikey_list if x not in sink0]

(492, 13)


492it [00:00, 6813.77it/s]


In [127]:
not_lipid_rxndb_inteme_only_one_new['reactant_inchikey0'].to_list()[0]

['PELZSPZCXGTUMR', 'XLYOFNOQVPJJNP']

In [128]:
for x in filter_inchi:
    if x not in sink0:
        print(x)

CJLVUWULFKHGFB
WSQWJAOQSDPYTD
KSFOVUSSGSKXFI
MRNZYUAGJLJQAM
NPSPRYXPOGPCPM
ZHLWCBHWYUISFY
PUEDDPCUCPRQNY
AYFXPGXAZMFWNH
LZEXYCAGPMYXLX
KYTPWZMUSLPBJZ
PIUSLWSYOYFRFR
CILXJJLQPTUUSS
WWUZIQQURGPMPG
YHHSONZFOIEMCP
OGCWVIVNTBZPBW
CBIDVWSRUUODHL
UFSCUAXLTRFIDC
IWFZHMQFZRLGDQ
ARJPPNFIEQKVBB
FONIWJIDLJEJTL
SOXXPQLIZIPMIZ
NUZLRKBHOBPTQV
MZJFVXDTNBHTKZ
OKXISSDNJVDCJE
AFENDNXGAFYKQO
JVGVDSSUAVXRDY
NGSWKAQJJWESNS
NMUOATVLLQEYHI
KKZFLSZAWCYPOC
IAJILQKETJEXLJ
PYOHERBGXSPHQI
MCKAJXMRULSUKI
DXWQLTOXWVWMOH
HHPDVQLBYQFYFA
VSSDMJHMYISZJU
AOKQNZVJJXPUQA
SERHXTVXHNVDKA
PJBQWWHYTVYMLO
SKCYVGUCBRYGTE
JJIHLJJYMXLCOY
LLLPDUXGHXIXIW
HGINCPLSRVDWNT
SXZYCXMUPBBULW
SXZYCXMUPBBULW
SNDPXSYFESPGGJ
DZTHIGRZJZPRDV
NUJGJRNETVAIRJ
WXNXCEHXYPACJF
GNMSLDIYJOSUSW
IPBVNPXQWQGGJP
MBPFNOMGYSRGQZ
ZKUSPPOKDDRMIU
PELZSPZCXGTUMR
KTHDTJVBEPMMGL
HRTOQFBQOFIFEE
RNDXVZITNYDUMC
DPJKHFICSGCNIR
IHYJTAOFMMMOPX
UHEPJGULSIKKTP
PMTUDJVZIGZBIX
DWPTUFGRYYIXHG
FSASIHFSFGAIJM
HSHNITRMYYLLCV
IPEHBUMCGVEMRF
XHZMOKNFPZDZBZ
FZWGECJQACGGTI
KABFMIBPWC

In [129]:
new_met

['PELZSPZCXGTUMR',
 'MCKAJXMRULSUKI',
 'ZAWIXNGTTZTBKV',
 'CAHKINHBCWCHCF',
 'UFSCUAXLTRFIDC',
 'FYSSBMZUBSBFJL',
 'JARKCYVAAOWBJS',
 'UFSCUAXLTRFIDC',
 'UFSCUAXLTRFIDC',
 'MCKAJXMRULSUKI',
 'PELZSPZCXGTUMR',
 'AFENDNXGAFYKQO',
 'JVGVDSSUAVXRDY',
 'ZCZXOHUILRHRQJ',
 'PTJWIQPHWPFNBW',
 'PTJWIQPHWPFNBW',
 'PTJWIQPHWPFNBW',
 'PTJWIQPHWPFNBW',
 'PTJWIQPHWPFNBW',
 'PTJWIQPHWPFNBW',
 'PTJWIQPHWPFNBW',
 'PTJWIQPHWPFNBW',
 'PTJWIQPHWPFNBW',
 'PTJWIQPHWPFNBW',
 'PTJWIQPHWPFNBW',
 'PUEDDPCUCPRQNY',
 'PUEDDPCUCPRQNY',
 'PUEDDPCUCPRQNY',
 'PUEDDPCUCPRQNY',
 'PUEDDPCUCPRQNY',
 'PUEDDPCUCPRQNY',
 'PUEDDPCUCPRQNY',
 'PUEDDPCUCPRQNY',
 'PUEDDPCUCPRQNY',
 'PUEDDPCUCPRQNY',
 'PUEDDPCUCPRQNY',
 'IPBVNPXQWQGGJP',
 'KSMVZQYAVGTKIV',
 'NUJGJRNETVAIRJ',
 'JARKCYVAAOWBJS',
 'VEYYWZRYIYDQJM',
 'YHHSONZFOIEMCP',
 'YHHSONZFOIEMCP',
 'YHHSONZFOIEMCP',
 'YHHSONZFOIEMCP',
 'YHHSONZFOIEMCP',
 'YHHSONZFOIEMCP',
 'YHHSONZFOIEMCP',
 'YHHSONZFOIEMCP',
 'YHHSONZFOIEMCP',
 'YHHSONZFOIEMCP',
 'YHHSONZFOIEMCP',
 'XQXPVVBIMD

In [130]:
len(set(new_met))

69

In [132]:
success_predict_met_num = 0
for x in tqdm(not_lipid_target_smiles_filter20percent_else):
    # print(smiles2inchikey0(x))
    if smiles2inchikey0(x) in set(new_met):
        success_predict_met_num+=1
success_predict_met_num

100%|██████████| 572/572 [00:00<00:00, 2241.39it/s]


73

In [120]:
break

SyntaxError: 'break' outside loop (668683560.py, line 4)

# check success_smiles in filter20percent_else meta

In [22]:
not_lipid_target_smiles_filter20percent_else_path = '../../../Results/not_lipid_filter20percent/target_smiles_not_lipid_filter20percent_else.pickle'
not_lipid_target_smiles_filter20percent_else = load_pickle(not_lipid_target_smiles_filter20percent_else_path)
len(not_lipid_target_smiles_filter20percent_else)

281

In [23]:
not_lipid_target_smiles_filter20percent_else_path = '../../../Results/not_lipid_filter20percent/target_smiles_not_lipid_filter20percent_else.pickle'
not_lipid_target_smiles_filter20percent_else = load_pickle(not_lipid_target_smiles_filter20percent_else_path)
filter_inchi = [smiles2inchikey0(x) for x in not_lipid_target_smiles_filter20percent_else]

In [24]:
num = 0
for x in not_lipid_target_smiles_filter20percent_else:
    if any(compare_smiles_inchikey(x,met) for met in success_smiles):
        num +=1
num

0

In [25]:
YMDB_success_met_smile_file = '../../../Results/not_lipid/top50_0.3_add_no_ec_re/YMDB_success_met_smile_top50_0.3.pickle'
YMDB_success_met_smile = load_pickle(YMDB_success_met_smile_file)
len(YMDB_success_met_smile)

221

In [ ]:
num = 0
for x in not_lipid_target_smiles_filter20percent_else:
    if any(compare_smiles_inchikey(x,met) for met in YMDB_success_met_smile):
        num +=1
num

100

In [27]:
29/34

0.8529411764705882